In [502]:
import csv
from datetime import datetime, timedelta
import re
import random
import time
import os
import logging
import sys
import errno
import traceback
from pathlib2 import Path
import copy

from bs4 import BeautifulSoup, NavigableString, Tag
import lxml.html
import requests

import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

print 'all done'

all done


In [84]:
firefox_win_ua = 'Mozilla/5.0 (Windows NT 6.1; rv:52.0) Gecko/20100101 Firefox/52.0'
firefox_mac_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:58.0) Gecko/20100101 Firefox/58.0'
safari_mac_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/604.4.7 (KHTML, like Gecko) Version/11.0.2 Safari/604.4.7'

domain = 'www.prisontalk.com'

headers =   {
                'Host': domain,
                'User-Agent': firefox_win_ua,
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.5',
                'Accept-Encoding': 'gzip, deflate',
                'DNT': '1',
                'Connection': 'keep-alive'
            }

## THREAD SCRAPER

**NOTE:** *A FEW FORUMS REQUIRE BEING LOGGED IN AND THEREFORE SO WILL THE THREADS OF FORUMS, THIS SCRAPER ONLY WORKS FOR THREADS THAT DO NOT REQUIRE LOGGING IN.  THESE THREADS WILL BE ADDRESS IN ANOTHER NOTEBOOK*

#### SETUP VARIABLES

In [497]:
# GET REMAINING FORUMS BY REMOVING ALREADY RETRIEVED FORUMS FROM FULL LIST
# remaining_forums = remove_retrieved_forums(full_forum_list, retrieved_fids)

debug = True

if debug:
    root = '../examples'
    min_time = 4
    max_time = 7
else:
    root = '../data'
    min_time = 8
    max_time = 18

in_path = root + '/threads/thread_lists/'
out_path = root + '/threads/'
log_path = root + '/threads/logs/'

# GET FILE PATHS TO ALL CSV LISTS OF THREADS
any_digit = re.compile(r'\d+')
forums = [str(path) for path in Path(in_path).iterdir() if path.is_file()]
forum_ids = [any_digit.findall(forum_dir)[0] for forum_dir in forums]

# print forum_ids
# print forums
# print get_next_scrape_times(min_time, max_time)

#### SCRAPE

In [500]:
# HOW LONG WE'LL INITIALLY SCRAPE FOR BEFORE TAKING A BREAK
start_time, scrape_time = get_next_scrape_times(min_time, max_time)
print 'SCRAPE TIME', scrape_time

for forum, forum_id in zip(forums, forum_ids):
    
    try:
        logger = create_forum_logger(log_path, forum_id)
        
        threads = get_thread_list(forum)

        for thread in threads:
        
            thread['thread_url'] = get_thread_url(thread['thread_id'], 0, 40)
            thread['num_pages'] = get_num_thread_pages(thread['thread_url'], headers)
            
            log_thread_report('scraping', thread)

            for page in range(thread['num_pages']):
                
                thread_page_url = get_thread_url(thread['thread_id'], page, 40)
                thread_page = s.get(thread_page_url, headers=headers)
                
                if not thread_page.raise_for_status():
                    
                    log_thread_report('retrieved', thread, page, response=thread_page)
                    save_html_file(thread_page, out_path, thread, page)
                                        
                    if time_to_take_a_break(start_time, scrape_time):

                        wait_time = random.randint(20, 30) * 60
                        send_progress_report('downtime', thread['forum_id'], wait_time)
                        start_time, scrape_time = get_next_scrape_times(min_time, max_time, wait_time, seconds=True)
                        print 'SCRAPE TIME', scrape_time
                    
                    else:
                        wait_time = random.randint(6, 10)

                    time.sleep(wait_time)

            log_thread_report('completed', thread, page)

        send_progress_report('forum_complete', forum_id)

    except Exception as err:
        
        report_sent = send_critical_report(err, traceback)

        raise

## FUNCTIONS

### SCRAPING

#### GET THREAD LIST

In [243]:
def get_thread_list(csv_filename):
    """
    Given a path to a csv file which lists all threads in a forum
    Returns a list of dicts
    Logs "LOADED X THREADS FROM: path/to/csv_filename.csv"
    
    Parameters
    ----------
    csv_filename : str
        The path and filename to the csv file
        
    Returns
    -------
    thread_list : list
        A list of dicts where each dict is one line of the csv file
        
    Examples
    --------
    >>> csv_filename = '../some/path/threads.csv'
    >>> thread_list = get_thread_list(csv_filename)
    >>> thread_list[0]
    
    {'forum_id': '65',
     'num_thread_replies': '127',
     'num_thread_views': '38,017',
     'thread_id': '77500',
     'thread_title': 'Welcome to the Juvenile Forum-Introduce yourself'}
    
    """
    
    thread_list = []
    
    with open(csv_filename) as csv_file:
        
        reader = csv.DictReader(csv_file)
        
        for thread in reader:
                        
            thread_list.append(thread)
            
    logger.info('LOADED ' + str(len(thread_list)) + ' THREADS FROM: ' + csv_filename)

    return thread_list

# csv_filename = '../examples/threads/thread_lists/thread_list_65.csv'
# thread_list = get_thread_list(csv_filename)

#### GET THREAD URL

In [103]:
def get_thread_url(thread_id, start_page, posts_per):
    
    """
    Given a thread id, starting page, and number of pages to show at a time, returns a URL
    
    Parameters
    ----------
    thread_id : int
        The desired thread to retrieve
    start_page : int
        The page in the thread to retrieve
    pages_per : int
        How many pages posts to display at a time
        
    Returns
    -------
    str
        A url to retrieve the page of posts of the thread requested
        
    Examples
    --------
    >>> get_thread_url(671868, 0, 40)
    'http://prisontalk.com/forums/showthread.php?t=671868&order=DESC&pp=40&page=1'
    
    """

    thread = str(thread_id)
    order = 'DESC'
    pp = str(posts_per) # THE MAX IS 40 FOR THREADS
    page = str(start_page + 1) # SINCE INEVITABLE ARRAY STARTS AT THE 0th ELEMENT, ADD 1

    thread_url = 'http://prisontalk.com/forums/showthread.php?'+ \
                                                't='+thread+ \
                                                '&order='+order+ \
                                                '&pp='+pp+ \
                                                '&page='+page
    return thread_url

# 'http://prisontalk.com/forums/showthread.php?t=671868&order=DESC&pp=40&page=1'
# get_thread_url(671868, 0, 40)

#### GET NUM THREAD PAGES

In [91]:
def get_num_thread_pages(thread_url, headers):
    """
    Given request headers and a URL to a thread in a forum, returns the number of pages of replies in the thread
    NOTE: Request session must already be started elsewhere
    
    Parameters
    ----------
    thread_url : str
        The URL location of page one of threads to retreive
    headers: dict
        The headers to be used for requesting the page over the web
        
    Returns
    -------
    num_pages : int
        The number of pages of replies in the thread
        
    Examples
    --------
    >>> thread_url = 'http://prisontalk.com/forums/showthread.php?t=671868&order=DESC&pp=40&page=1'
    >>> headers = {'Host': 'domain.com', ... 'Connection': 'keep-alive'}
    >>> 'NO. OF PAGES:', get_num_thread_pages(thread_url, headers)
    NO. OF PAGES: 4
    """
    
    try:
        initial_thread_page = s.get(thread_url, headers=headers)
        
        thread_page = str(BeautifulSoup(initial_thread_page.content, 'html.parser'))
        no_literals = thread_page.replace('\n', '').replace('\t', '').replace('\r', '')
        thread_page = BeautifulSoup(no_literals, 'html.parser')# skip u'html PUBLIC "-//W3C//
    
        # THERE IS A TOP AND BOTTOM NAV SO WE'LL JUST TAKE THE TOP
        nav = thread_page.find_all('div', {'class':'pagenav'})[0]
        page_x_of_y = nav.find_all('td', {'class':'vbmenu_control'})[0]
        num_pages = page_x_of_y.text.split(' ')[-1]
        
    except IndexError as er:
        
        num_pages = 1
    
    return int(num_pages)

# url = get_thread_url(671868, 0, 40)
# get_num_thread_pages(url, headers) # 4

#### SAVE HTML FILE

In [364]:
def save_html_file(thread_page, path, thread, page):
    """
    Given a http response, path, thread dict, and page number
    Saves response.contents as dirpath/threads/fid/fid_X_tid_Y_pg_Z.html
    Logs MM/DD/YYYY HH:MM:SS PM INFO: SAVED: dirpath/threads/fid/fid_X_tid_Y_pg_Z.html
    
    Parameters
    ----------
    thread_page : response
        The response from a http request
    path : str
        The path to save the page to
    thread: dict
        A dict that contains at least the following keys: forum_id, thread_id
    page : int
        The number of the current page retrieved in the thread

    Returns
    -------
    forum_path + file_name : str
        The path and filename of the file saved
        
    Examples
    --------
    >>> save_html_file(response, 'path/to/dir/', thread, 0)
    """
    
    fid = str(thread['forum_id'])
    tid = str(thread['thread_id'])
    pg = str(page + 1)
    
    # NEW DIRECTORY PATH, EX: 'data/forums/28/'
    forum_path = path + fid + '/'
    # NEW FILENAME, EX: 'fid_39_pg_14.html 
    file_name = 'fid_' + fid + '_tid_' + tid + '_pg_' + pg + '.html'
    
    # CREATE FORUM_PATH DIRECTORY IF IT DOESN'T EXIST, ELSE, MOVE ON
    Path(forum_path).mkdir(parents=False, exist_ok=True)

    #SAVE FILE
    with open(forum_path + file_name, 'w') as html_file:

        html_file.write(thread_page.content)
        
    logger.info('SAVED ' + forum_path + file_name)

# save_html_file(thread_page, out_path, threads[0], 0)
# # '../examples/threads/65/fid_65_tid_77500_pg_1.hmtl'

#### TIME TO TAKE A BREAK

In [453]:
def time_to_take_a_break(start_time, minutes_to_scrape):
    """
    Given the time scraping had begun, and the number of minutes for which scraping should go on for
    Obtains the difference between the two
    Returns True if enough time has elapsed (difference is zero)
    Returns False if there is still time to go, (difference is greater than zero)
    
    Parameters
    ----------
    start_time : datetime.datetime
        Time the scraping began
    minutes_to_scrape : int
        The time in minutes the scraping should last
        
    Returns
    -------
    elapsed_time.total_seconds() >= seconds_to_scrape : bool
        True if elapsed time exceeds the time in seconds that scraping was to take place for
        
    Examples
    --------
    >>> five_seconds = 0.08343145
    >>> eight_seconds = 0.1341245
    >>> start = datetime.now()
    >>> time.sleep(5)
    >>> time_to_take_a_break(start, five_seconds), time_to_take_a_break(start, eight_seconds)
    (True, False)
    """
    
    current_time = datetime.now()
    # NOT REALLY NECCESSARY IN MINUTE CONDITIONS BUT TESTS ARE IN SECONDS AND I'M IMPATIENT
    seconds_to_scrape = int(minutes_to_scrape * 60.0) 
    
    elapsed_time = current_time - start_time
    break_time = elapsed_time.total_seconds() >= seconds_to_scrape
    # print 'BREAKTIME:', break_time, '--> SECONDS ELAPSED:', elapsed_time.total_seconds(), 'SECONDS TO SCRAPE:', seconds_to_scrape
    
    return break_time

# five_seconds = 0.08343145 # ~5.005 seconds, returns True
# eight_seconds = 0.1341245 # ~8.047 seconds, returns False
# start = datetime.now()
# time.sleep(5)
# time_to_take_a_break(start, five_seconds), time_to_take_a_break(start, eight_seconds) # (True, False)

#### GET NEXT SCRAPE TIMES

In [491]:
def get_next_scrape_times(min_time, max_time, additional_time=0, seconds=False):
    """
    Given a minimum and maximum time in minutes, returns the current time, 
    and a random length of time in minutes to scrape for.
    Used for calculating the difference between the two function time_to_take_a_break()
    Additionally, you can factor in some additional_time to compensate (e.g. time spent sleeping)
    
    Parameters
    ----------
    min_time : int
        Minimum time to scrape for in minutes
    max_time : int
        Maximum time to scrape for in minutes
    additional_time : int
        Optional - An ammount of time to add in minutes if seconds is False (see below), default is zero 
    seconds : boolean
        Optional - What time scale the additional_time is in, default is False (e.g. minutes)
        
    Returns
    -------
    datetime.datetime
        The current time plus any additional time
    int
        The time to scrape for in minutes
        
    Examples
    --------
    >>> start_time, scrape_time = get_next_scrape_times(8, 18, 3)
    >>> print 'WAIT:', wait_time, '\n', 'NOW:', datetime.now(), '\n', 'BEGIN NEXT SCRAPE:', start_time, '\n', 'NEXT SCRAPE LENGTH:', scrape_time, 'minutes'
    WAIT: 3 
    NOW: 2018-06-20 13:36:14.822685 
    BEGIN NEXT SCRAPE: 2018-06-20 13:39:14.821660 
    NEXT SCRAPE LENGTH: 17 minutes
    """
    
    wait_time = additional_time * 60 if seconds == False else additional_time
    
    next_start_time = datetime.now() + timedelta(seconds=wait_time) # TO SUBTRACT LATER
    next_scrape_length = random.randint(min_time, max_time) # HOW LONG WE'LL SCRAPE FOR BEFORE TAKING A BREAK

    return next_start_time, next_scrape_length

# wait_time = random.randint(6, 10)
# start_time, scrape_time = get_next_scrape_times(8, 18, wait_time)
# print 'WAIT:', int(wait_time), 'minutes' '\n', 'NOW:', datetime.now(), '\n', 'BEGIN NEXT SCRAPE:', start_time, '\n', 'NEXT SCRAPE LENGTH:', scrape_time, 'minutes'
# print ''
# wait_time = random.randint(6, 10) * 60
# start_time, scrape_time = get_next_scrape_times(8, 18, wait_time, seconds=True)
# print 'WAIT:', int(wait_time / 60.0), 'minutes' '\n', 'NOW:', datetime.now(), '\n', 'BEGIN NEXT SCRAPE:', start_time, '\n', 'NEXT SCRAPE LENGTH:', scrape_time, 'minutes'

### REPORTING

#### CREATE FORUM LOGGER

In [337]:
def create_forum_logger(file_path, forum_id):
    """
    Given a file path and thread dict 
    Returns a logger which logs to that path with the file forum_X.log in the following format
    18/06/2018 05:27:11 PM INFO: SOME TEST MSG
    
    Parameters
    ----------
    file_path : str
        The path to where the log should be written to
    forum_id : int
        The interger id of the your are going to scrape

    Returns
    -------
    logger : logger
        The logger which saves logs to the file forum_X.log
        
    Examples
    --------
    >>> logger = create_forum_logger('some/path/to/log/', thread_dict)
    >>> logger.info('SOME TEST MSG')
    """

    log_file = 'forum_'+ str(forum_id) + '.log'
    msgfmt = '%(asctime)s %(levelname)s: %(message)s'
    datefmt= '%d/%m/%Y %I:%M:%S %p'
    
    reload(logging)
    logging.basicConfig(filename=file_path + log_file, level=logging.INFO, format=msgfmt, datefmt=datefmt)
    logging.Formatter.converter = time.gmtime # CHANGE TO UTC TIME
    logger = logging.getLogger()
    
    return logger 

# log_path = '../examples/threads/logs/'
# fid = 0
# logger = create_forum_logger(log_path, fid)
# logger.info('SOME TEST MSG') # logs 18/06/2018 05:27:11 PM INFO: SOME TEST MSG to forum_0.log

#### LOG THREAD REPORT

In [303]:
def log_thread_report(status, thread, current_page=0, response=None):
    """
    Given a thread dict and a status type
    Logs a status report to the thread log
    18/06/2018 05:27:11 PM INFO: RETRIEVING FORUM: 65 THREAD: 77500 REPLIES: 122 PAGE: 0 OF 4
    
    Parameters
    ----------
    status: str
        A string which indicates which message to log: 'retreiving', 'completed'
    thread : dict
        A dict that includes the keys: forum_id, thread_id, num_pages, num_thread_replies
    current_page : int
        Current page being retreived
    *args : Added parameters for various messages
    
    Returns
    -------
        None
    
    Examples
    --------
    >>> log_thread_report('scraping', thread, page)
    """
    
    fid = str(thread['forum_id'])
    tid = str(thread['thread_id'])
    rpls = str(thread['num_thread_replies'])
    pg = str(current_page + 1)
    pgs = str(thread['num_pages'])
    
    if response:
        query = response.url.split('/')[-1]
        status_code = str(response.status_code)

    if status == 'scraping':
        report = status.upper()+' FORUM: '+fid+' THREAD: '+tid+ ' REPLIES: '+rpls+' PAGES: '+pgs
    elif status == 'retrieved':
        report = status.upper()+' THREAD: '+tid+' PAGE: '+pg+' OF '+pgs+' FROM: '+query+' STATUS: '+status_code
    elif status == 'completed':
        report = status.upper()+' SCRAPING '+pg+' OF '+pgs+' PAGES FROM FORUM: '+fid+' THREAD: '+tid+ ' REPLIES: '+rpls
        
    logger.info(report)
    # print report

# logger = create_forum_logger(log_path, threads[0])
# # 18/06/2018 06:25:18 PM INFO: RETREIVING FORUM: 65 THREAD: 77500 REPLIES: 127 PAGE: 3 OF 4
# url_str = 'http://prisontalk.com/forums/showthread.php?t=671868&order=DESC&pp=40&page=1'
# test_page = s.get(url_str, headers=headers)
# log_thread_report('scraping', threads[0])
# log_thread_report('retrieved', threads[0], 0, response=test_page)
# log_thread_report('completed', threads[0], 3)

#### SEND PROGRESS REPORT

In [496]:
def send_progress_report(progress, forum_id, seconds=0, total_forums=0, text=False):
    
    try:

        if progress == 'forum_complete':

            headline = 'FINISHED SCRAPING LIST OF THREADS FROM FORUM ' + str(forum_id) #+ '\r\n'
            other_info = '' # MAYBE ADD HOW MANY FORUMS ARE LEFT?
            closing = '\r\n' + 'TALK SOON!'

        elif progress =='downtime':

            headline = 'TAKING A BREAK FOR ' + str(int(seconds/60.0)) + ' MINUTES TO KEEP UNDER THE RADAR' #+ '\r\n'
            other_info = '\r\n'+ 'STILL DOWNLOADING THREADS FROM FORUM ' + str(forum_id) #+ '\r\n'
            closing = '\r\n' + 'BE SEEING YOU...!'

        elif progress == 'finished':

            headline = 'IT IS FINISHED: ALL ' + str(total_forums) + ' LISTS OF THREADS DOWNLOADED' #+ '\r\n'
            other_info = '\r\n' + 'LAST FORUM DOWNLOADED: ' + str(forum_id) #+ '\r\n'
            closing = '\r\n' +'I\'M PRETTY CACHED, AND GRATEFUL WE DIDN\'T GET BANNED. PEACE.'

        else:

            headline = 'AN UNKNOWN PROGRESS STATUS WAS PASSED: ' + progress #+ '\r\n'
            other_info = '\r\n' + 'CONTINUING TO DOWNLOAD THREADS FROM FORUM: ' + str(forum_id) #+ '\r\n'
            closing = '\r\n' + 'SORRY.'

        logger.info(headline)

        email_msg = MIMEMultipart()

        email_msg['From']=' YOUR SCRAPER'
        email_msg['To'] = 'YOUR EMAIL ADDRESS'
        email_msg['Subject'] = 'PRISONTALK: PROGRESS REPORT'
        email_msg.attach(MIMEText(headline + other_info + closing, 'plain'))

        smtp_gmail = smtplib.SMTP('smtp.gmail.com', 587)
        smtp_gmail.starttls()
        smtp_gmail.login('YOUR GMAIL EMAIL ADDRESS', 'YOUR GMAIL EMAIL PASSWORD')

        smtp_gmail.sendmail(email_msg['From'], email_msg['To'], email_msg.as_string())

        if text:

            text_msg = MIMEMultipart()

            text_msg['From']= 'YOUR SCRAPER'
            text_msg['To'] = 'YOUR PHONE NUMBER'
            text_msg['Subject'] = 'GOOD NEWS EVERYONE!'
            text_msg.attach(MIMEText(progress.upper()))

            smtp_gmail.sendmail(text_msg['From'], text_msg['To'], text_msg.as_string())

        logger.info('PROGRESS REPORT SENT VIA ' + ('TEXT & EMAIL' if text == True else 'EMAIL') + '!')

        smtp_gmail.close()
        
    except Exception as err:
        
        err_type = err.__class__.__name__
        
        logger.info('COULDN\'T SEND PROGRESS REPORT:', err_type, err) 
        logger.info('TRIED TO REACH YOU. GOT VOICEMAIL. I THINK I\'LL JUST QUIT WHILE I\'M AHEAD')
        
        return False 


# forum_id = 65
# seconds = 600
# list_of_forums = [65, 105, 122, 320]

# send_progress_report('downtime', forum_id, seconds)
# send_progress_report('forum_complete', forum_id, text=True)
# send_progress_report('finished', forum_id, total_forums=len(list_of_forums), text=True)
# send_progress_report('continuing', forum_id, text=True)

#### SEND CRITICAL FAILURE REPORT

In [318]:
def send_critical_report(err, tb):
    
    try:
        
        logger.info('CRITICAL ERROR: ' + err.__class__.__name__ + ' ' + str(err))

        text_msg = MIMEMultipart()

        text_msg['From']=' YOUR SCRAPER'
        text_msg['To'] = 'YOUR PHONE NUMBER'
        text_msg['Subject'] = ' DON\'T PANIC BUT...'
        text_body = 'CHECK YOUR EMAIL.'

        text_msg.attach(MIMEText(text_body))

        
        email_msg = MIMEMultipart()

        email_msg['From']=' YOUR SCRAPER'
        email_msg['To'] = 'YOUR EMAIL ADDRESS'
        email_msg['Subject'] = 'PRISONTALK: CRITICAL ERROR REPORT'
        
        tb_str = tb.format_exc() # REMOVE IN THE FUTURE AND ADD DIRECTLY TO BODY OR KEEP FOR CLARITY?
        email_body = 'ERROR:\r\n' + \
                    '\r\n' + \
                    tb_str + \
                    '\r\n' + \
                    'TRIED TO REACH YOU.\r\n' + \
                    'GOT VOICEMAIL.\r\n' + \
                    'GET READY FOR THE THUNDERDOME.\r\n' + \
                    'SHUT IT DOWN FOLKS. SHUT IT DOWN.'

        email_msg.attach(MIMEText(email_body, 'plain'))


        smtp_gmail = smtplib.SMTP('smtp.gmail.com', 587)
        smtp_gmail.starttls()
        smtp_gmail.login('YOUR GMAIL EMAIL ADDRESS', 'YOUR GMAIL EMAIL PASSWORD')

        smtp_gmail.sendmail(text_msg['From'], text_msg['To'], text_msg.as_string())
        smtp_gmail.sendmail(email_msg['From'], email_msg['To'], email_msg.as_string())
        
        smtp_gmail.close()
        logger.info('CRITICAL ERROR: REPORT SENT VIA TEXT & EMAIL')
        
        return True
        
    except Exception as err:
                
        logger.info('DOUBLE CRITICAL ERROR: ' + err.__class__.__name__ + ' ' + str(err)) 
        logger.info('TRIED TO REACH YOU. GOT VOICEMAIL. SHUT IT DOWN FOLKS! SHUT IT DOWN.')
        
        return False 

# # TEST IN A LOOP TO MAKE SURE IT WILL EXIT AND YOU WON'T BE SPAMMED BY YOUR OWN PROGRAM
# for i in range(2):
    
#     try:
#         1/0
#     except Exception as err:

#         report_sent = send_critical_report(err, traceback)
#         print 'REPORT SENT:', report_sent

#         raise        

# # SHOULD PRINT THE FOLLOWING AND SEND *ONE* TEXT AND EMAIL OF MESSAGE ABOVE
# # REPORT SENT: True
# # ---------------------------------------------------------------------------
# # ZeroDivisionError                         Traceback (most recent call last)
# # ...

### HELPERS

#### GET RETRIEVAL TIME

In [109]:
def get_retrieval_time(replies, replies_per_page):
    """
    Given a number of threads and the threads per page
    Returns an the time in minutes of how long it will take to download all of the pages
    
    Parameters
    ----------
    replies : str
        The number of threads
    replies_per_page : int
        The number of threads on each page
        
    Returns
    -------
    total_minutes : int
        The total number of minutes it will take to download the pages of replies
        
    Examples
    --------
    >>> get_retrieval_time(4440, 40)
    12
    """
    
    average_page_wait = 7
    
    pages = int(replies)/replies_per_page
    
    total_seconds = pages * 7
    
    total_minutes = total_seconds/60
    
    return total_minutes

# get_retrieval_time(4440, 40) # 12

#### PRINT HEADERS

In [98]:
def print_headers(response):
    
    print 'RESPONSE HEADERS'
    for key, val in response.headers.iteritems():
        print key + ':', val
    print ''
    print 'REQUEST HEADERS'
    for key, val in response.request.headers.iteritems():
        print key + ':', val
        
# s = requests.session()
# test_response = s.get('http://prisontalk.com', headers=headers)
# print_headers(test_response)

# # RESPONSE HEADERS
# # Date: Mon, 26 Feb 2018 20:57:24 GMT
# # Server: Apache/2.2.3 (CentOS)
# # ...
# # Transfer-Encoding: chunked
# # Content-Type: text/html; charset=ISO-8859-1
    
# # REQUEST HEADERS
# # Connection: keep-alive
# # Accept-Encoding: gzip, deflate
# # ...
# # DNT: 1
# # Host: www.prisontalk.com

#### EXCEPTIONS TESTING

In [349]:
# for i in [0,1,2]:
    
#     try:
#         if i == 1:
#             1/0
#         else: 
#             print 'i is equal to', i
        
#     except Exception as err:

# #         print err.__class__.__name__, err
        
# #         print traceback.format_exc()

#         raise